In [18]:
import os
os.chdir(r"C:/Users/Admin/Desktop/Kristal/advisory_vols_gen")
import pandas as pd
import numpy as np

In [21]:
kristals_list = pd.read_csv("List of Assets.csv")
kristals_list = kristals_list.rename(columns={'Normal Kristals' : 'kristal_name'})

In [22]:
# Putting labels for single & multi asset kristals
kristals_list.loc[(pd.notnull(kristals_list['symbol'])), 'kristal_type'] = 'single_asset'
kristals_list.loc[(pd.isnull(kristals_list['symbol'])), 'kristal_type'] = 'multiple_assets'

In [23]:
# Joining with model accounts file to get asset IDs & asset weights for each kristal
model_accounts = pd.read_excel("model_account_position.xlsx")
comb = kristals_list.merge(model_accounts, how='left', on='kristal_name')
assets_df = comb[['kristal_name', 'asset_id', 'kristal_type', '% in kristal']] 
assets_df = assets_df.rename(columns={'% in kristal' : 'asset_weight'})

# Drop kristal whose asset is not found in model_accounts 
assets_df.drop(assets_df[pd.isnull(assets_df.asset_id)].index, inplace=True)

In [24]:
# Joining with assets_bbg file to get PROXY_INDEX and VOL for each asset
assets_bbg = pd.read_excel("assets_bbg.xlsx", sheet_name="Assets")
comb2 = assets_df.merge(assets_bbg, how='left', on='asset_id')
assets_vol = comb2[['kristal_name', 'asset_id', 'asset_weight', 'kristal_type', 'PROXY_INDEX', 'VOL']]
assets_vol = assets_vol.drop_duplicates()

# Printing out assets file if some assets are not found in assets_bbg 
assets_nf = assets_vol.loc[pd.isnull(assets_vol.asset_id)]
if (len(assets_nf) > 0):
    print("Some assets not found in assets_bbg ... printing these ...")
    assets_nf.to_csv('assets_not_found.csv', index=False)

# asset_id = 508: drop for single-asset kristals & put vol = 0 for multi-asset kristal
assets_vol.drop(assets_vol[(assets_vol.asset_id == 508) & (assets_vol.kristal_type == 'single_asset')].index, inplace=True)
assets_vol.loc[((assets_vol.asset_id == 508) & (assets_vol.kristal_type == 'multiple_assets')), 'VOL'] = 0

In [26]:
assets_vol.to_csv('assets_vol.csv', index=False)